<a href="https://colab.research.google.com/github/st3082group10/Abalone_Age_Prediction/blob/main/Abolone_Random_Forest_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Reading the cleaned numeric abolone data set
import pandas as pd
import numpy as np
 
# To remove the scientific notation from numpy arrays
np.set_printoptions(suppress=True)
 
from google.colab import files
uploaded = files.upload()
import io
from sklearn.preprocessing import LabelEncoder 
 
aboloneAgeDataNumeric = pd.read_csv(io.BytesIO(uploaded['abalone.csv']))

# Remove rows where height is equal to 0
aboloneAgeDataNumeric = aboloneAgeDataNumeric[aboloneAgeDataNumeric['Height'] != 0]

# Create an instance of LabelEncoder
le = LabelEncoder()

# Encode the "Sex" variable
aboloneAgeDataNumeric["Sex"] = le.fit_transform(aboloneAgeDataNumeric["Sex"])

aboloneAgeDataNumeric.head()


Saving abalone.csv to abalone.csv


,Sex,Length,Diameter,Height,Whole weight,Shucked weight,Viscera weight,Shell weight,Rings
0,1,0.455,0.365,0.095,0.5140,0.2245,0.1010,0.150,15
1,1,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.070,7
2,0,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.210,9
3,1,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.155,10
4,2,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.055,7


In [2]:
# Separate Target Variable and Predictor Variables
TargetVariable=['Rings']
Predictors=['Sex', 'Length', 'Diameter', 'Height', 'Whole weight','Shucked weight','Viscera weight', 'Shell weight']

X=aboloneAgeDataNumeric[Predictors].values
y=aboloneAgeDataNumeric[TargetVariable].values

### Sandardization of data ###
from sklearn.preprocessing import StandardScaler
PredictorScaler=StandardScaler()
TargetVarScaler=StandardScaler()

# Storing the fit object for later reference
PredictorScalerFit=PredictorScaler.fit(X)
TargetVarScalerFit=TargetVarScaler.fit(y)

# Generating the standardized values of X and y
X=PredictorScalerFit.transform(X)
y=TargetVarScalerFit.transform(y)

# Set the random seed
np.random.seed(123)

# Split the data into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

# Quick sanity check with the shapes of Training and testing datasets
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(3340, 8)
(3340, 1)
(835, 8)
(835, 1)


In [9]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor

y_train = y_train.ravel()
y_test = y_test.ravel()

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

# Define the hyperparameter grid to search over
param_grid = {
    'n_estimators': [10, 50, 100, 200, 500],
}

# Create a random forest model
rf = RandomForestRegressor()

# Perform a grid search with cross-validation
grid_search = GridSearchCV(rf, param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(X_train, y_train)

# Print the best hyperparameters and corresponding validation score
print("Best parameters:", grid_search.best_params_)
print("Validation score:", -grid_search.best_score_)


Best parameters: {'n_estimators': 500}
Validation score: 0.45780229569738395


In [10]:
# create a new random forest model with the best hyperparameters
best_rf_model = RandomForestRegressor(n_estimators=500, max_depth=10, min_samples_split=10, random_state=123)

# train the model on the entire training dataset
best_rf_model.fit(X_train, y_train)

# make predictions on the test dataset
y_train_pred = best_rf_model.predict(X_train)

# make predictions on the test dataset
y_test_pred = best_rf_model.predict(X_test)


In [11]:
#Compute the MSE for training and testing
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

train_mse = mean_squared_error(y_train, y_train_pred) 
test_mse = mean_squared_error(y_test, y_test_pred) 

#Compute the R-squared for training and testing
train_r2 = r2_score(y_train, y_train_pred) 
test_r2 = r2_score(y_test, y_test_pred)

#Compute the MAPE for training and testing
mape = np.mean(np.abs((y_test - y_test_pred) / y_test)) * 100 

#Print out the results
print('Training MSE:', train_mse) 
print('Testing MSE:', test_mse)
print('Training R-squared:', train_r2) 
print('Testing R-squared:', test_r2) 
print('Mean Absolute Percentage Error:', round(mape, 2))

Training MSE: 0.21385851152514954
Testing MSE: 0.4197322033308361
Training R-squared: 0.7895935318982537
Testing R-squared: 0.5505819396781473
Mean Absolute Percentage Error: 308.87
